<a href="https://colab.research.google.com/github/amfakh/eapanicobacoba/blob/master/Scrape_Genius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sauce:

https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2

https://www.promptcloud.com/blog/scraping-song-lyrics-using-python-from-genius/

In [1]:
import urllib.request
import urllib.parse
import urllib.error
import ssl
import os
import re
import requests

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
GENIUS_API_TOKEN='PXpvxuPl_XyMM02ly-gyw8bd7tyP1avUQLZVsR4ooRmhfbTDOWG8mEHez0DGygjp'

In [3]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response# Get Genius.com song url's from artist object
    
# DEMO
# request_song_url('JKT48', 10)

In [4]:
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs

In [5]:
# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [6]:
song_json = {}
song_json["Primary Artist"] = []
song_json["Song Title"] = []
song_json["Lyrics"] = []


def scrape_from_link(url):

    req = Request(url, headers = { 'User-Agent' : 'Mozilla/5.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')

    for div in soup.findAll('h1', attrs = {'class': 'header_with_cover_art-primary_info-title'}):
        song_json["Song Title"].append(div.text.strip());
        print("Scrapped: " + song_json["Song Title"][-1])

    for div in soup.findAll('a', attrs = {'class': 'header_with_cover_art-primary_info-primary_artist'}):
        song_json["Primary Artist"].append(div.text.strip());    

    for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
        song_json["Lyrics"].append(div.text.strip().split("\n"));

    # print(song_json["Song Title"])


In [7]:
# scrape_from_link('https://genius.com/Jkt48-only-today-lyrics')

In [8]:
# song_json

In [17]:
import json

def write_lyrics_to_file(artist_name, song_count):
    # song_json["Primary Artist"].clear()
    # song_json["Song Title"].clear()
    # song_json["Lyrics"].clear()

    with open(artist_name.lower() + '.json', 'w') as outfile:
        urls = request_song_url(artist_name, song_count)
        for url in urls:
            # print(url)
            scrape_from_link(url)
      
        json.dump(song_json, outfile, indent = 4, ensure_ascii = False)
  
# DEMO  
write_lyrics_to_file('Fiersa Besari', 50)

Found 50 songs by Fiersa Besari
Waktu Yang Salah
Celengan Rindu
April
Bukan Lagu Valentine
Cerita Panjang Dihidup yang Singkat
Pelukku Untuk Pelikmu (OST Imperfect: Karier, Cinta, & Timbangan)
Sepasang Pendaki
Hidup Kan Baik-Baik Saja
Rumah
Tempat Aku Pulang
Ibu
Anak Semua Bangsa
Tiffa
Friendzone
Nona Senja
Edelweiss
Napak Tilas
Konspirasi Alam Semesta
Juara Kedua
Senja Bersayap
Hingga Napas Ini Habis
Acak Corak
Kau
I Heart Thee
Melangkah Tanpamu
Home
Lads
Hingga Akhir Nafas Ini
Glimpse
Kala
Hari Untukmu
Epilog
Lekas Pulih
Harapan
Kisah Semu
Temaram
Samar
And I Need You
Terima Kasih dan Maaf
Salahkah Mengalah
Petualangan
Selamat Tinggal
Kelana
Lembayung
Tanpa Karena
Nadir
Garis Terdepan
Telapak Kaki
Kawan Yang Mengagumkan
Bandung


In [10]:
import pandas as pd

In [15]:
df = pd.read_json(r'fiersa besari.json')

In [16]:
df

,Primary Artist,Song Title,Lyrics
0,Fiersa Besari,Waktu Yang Salah,"[[Lirik ""Waktu Yang Salah""], , [Verse 1: Fiers..."
1,Fiersa Besari,Celengan Rindu,"[[Verse 1], Aku kesal dengan jarak yang sering..."
2,Fiersa Besari,April,"[[Verse 1], Coba tanya hatimu sekali lagi, Seb..."
3,Fiersa Besari,Bukan Lagu Valentine,"[[Verse 1], Hei, gadis bermata cokelat, Di hat..."
4,Fiersa Besari,Cerita Panjang Dihidup yang Singkat,"[[Verse 1], Telah kita lalui perjalanan, Penuh..."
5,Fiersa Besari,"Pelukku Untuk Pelikmu (OST Imperfect: Karier, ...","[[Lirik ""Pelukku Untuk Pelikmu""], , [Verse 1],..."
6,Fiersa Besari,Sepasang Pendaki,"[[Verse 1], Kita melangkah susuri hutan berdua..."
7,Fiersa Besari,Hidup Kan Baik-Baik Saja,"[[Verse 1], Cerita kita tak semanis dongeng at..."
8,Fiersa Besari,Rumah,"[[Verse 1], Ratapanmu mengiringi kepergian kal..."
9,Fiersa Besari,Tempat Aku Pulang,"[[Verse 1], Apa kabarmu tambatan hati? Masihka..."
